# **Practice for DLRM PyTorch**

## **1. Installation**



In [ ]:
!nvidia-smi

Mon Aug  7 08:32:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install torch
!apt-get install unzip
!pip3 install pandas implicit sklearn onnx "git+https://github.com/mlperf/logging.git" surgeon-pytorch pandas scipy multiprocess
!git clone https://github.com/facebookresearch/dlrm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  Cloning https://github.com/mlperf/logging.git to /tmp/pip-req-build-wmw0lsu5
  Running command git clone --filter=blob:none --quiet https://github.com/mlperf/logging.git /tmp/pip-req-build-wmw0lsu5
  Resolved https://github.com/mlperf/logging.git to commit 9c93326b5b9f5893d40240aea90522044afbd722
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 48.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰

In [ ]:
import torch
print("import succeed")

import succeed


## **2. DLRM PyTorch**
- DLRM Paper : https://arxiv.org/abs/1906.00091


In [ ]:
cd dlrm

/content/dlrm


- Example model architecture :
<br/>
<img src="https://drive.google.com/uc?id=1Dc-ecvQ_4BH-7Y_H4WgaHmR8tFo3yPqq" width=350>

In [ ]:
!python dlrm_s_pytorch.py --arch-sparse-feature-size=16 --arch-embedding-size="100-100" --arch-mlp-bot="13-64-16" --arch-mlp-top="16-8-4-1" \
                          --data-generation random --mini-batch-size=2 --data-size=10 \
                          --use-gpu

Unable to import mlperf_logging,  No module named 'mlperf_logging'
2023-08-07 08:32:50.848799: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 08:32:51.790898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Unable to import onnx.  No module named 'onnx'
world size: 1, current rank: 0, local rank: 0
Using 1 GPU(s)...
time/loss/accuracy (if enabled):
Finished training it 1/5 of epoch 0, -1.00 ms/it, loss 0.207687
Finished training it 2/5 of epoch 0, -1.00 ms/it, loss 0.091614
Finished training it 3/5 of epoch 0, -1.00 ms/it, loss 0.073226
Finished training it 4/5 of epoch 0, -1.00 ms/it, loss 0.039275
Finished training it 5/5 of epoch 0, -1.00 ms/it, loss 0.009234


In [ ]:
import dlrm_s_pytorch as s
import numpy as np

dlrm = s.DLRM_Net(
    m_spa=16,
    ln_emb=np.array([100,100]),
    ln_bot=np.array([13,64,16]),
    ln_top=np.array([19,16,8,4,1]),
    arch_interaction_op="dot", # {"dot" / "cat"}
    arch_interaction_itself=False,
    qr_flag=False,
    md_flag=False,
)

dlrm

Unable to import mlperf_logging,  No module named 'mlperf_logging'
Unable to import onnx.  No module named 'onnx'


DLRM_Net(
  (emb_l): ModuleList(
    (0-1): 2 x EmbeddingBag(100, 16, mode='sum')
  )
  (bot_l): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=16, bias=True)
    (3): ReLU()
  )
  (top_l): Sequential(
    (0): Linear(in_features=19, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=4, bias=True)
    (5): ReLU()
    (6): Linear(in_features=4, out_features=1, bias=True)
    (7): ReLU()
  )
  (loss_fn): BCELoss()
)

- Input data format :
<br/>
<img src="https://drive.google.com/uc?id=1JGJMtMFa9gw0Zy_GFak7Bvc5tY3kcc8y" width=600>


In [ ]:
dense_x = torch.rand(2, 13)
sparse_offsets = [torch.LongTensor([0,4]), torch.LongTensor([0,4])]
sparse_indices = [torch.LongTensor([9,23,29,62,31,42,43,49]), torch.LongTensor([50,62,88,93,31])]

dlrm(dense_x, sparse_offsets, sparse_indices)

tensor([[1.1729],
        [1.3172]], grad_fn=<ReluBackward0>)

## **3. Memory-efficient embedding techniques**

### **Quotient Remainder Trick**
- Paper : https://arxiv.org/pdf/1909.02107.pdf



In [ ]:
import tricks.qr_embedding_bag as qr
qr_emb = qr.QREmbeddingBag(num_categories=100, embedding_dim=16, num_collisions=3)

qr_emb.weight_q.shape, qr_emb.weight_r.shape

(torch.Size([34, 16]), torch.Size([3, 16]))

In [ ]:
print(sparse_indices[0])
print(sparse_offsets[0])
print(qr_emb(sparse_indices[0], sparse_offsets[0]).shape)

tensor([ 9, 23, 29, 62, 31, 42, 43, 49])
tensor([0, 4])
torch.Size([2, 16])


- Run DLRM with `QREmbeddingBag`
  - QR Trick applied to embedding table which row size is bigger than `--qr-threshold`

In [ ]:
!python dlrm_s_pytorch.py --arch-sparse-feature-size=16 --arch-embedding-size="100-100" --arch-mlp-bot="13-64-16" --arch-mlp-top="16-8-4-1" \
                          --data-generation random --mini-batch-size=2 --data-size=10 \
                          --use-gpu \
                          --qr-flag --qr-threshold 50 --qr-operation mult --qr-collisions 3

Unable to import mlperf_logging,  No module named 'mlperf_logging'
2023-08-07 08:33:42.628286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Unable to import onnx.  No module named 'onnx'
world size: 1, current rank: 0, local rank: 0
Using 1 GPU(s)...
time/loss/accuracy (if enabled):
Finished training it 1/5 of epoch 0, -1.00 ms/it, loss 0.095611
Finished training it 2/5 of epoch 0, -1.00 ms/it, loss 0.227742
Finished training it 3/5 of epoch 0, -1.00 ms/it, loss 0.080723
Finished training it 4/5 of epoch 0, -1.00 ms/it, loss 0.199807
Finished training it 5/5 of epoch 0, -1.00 ms/it, loss 0.693947


### **Mixed Dimension Trick**
- Paper : https://arxiv.org/pdf/1909.11810.pdf

<img src="https://drive.google.com/uc?id=1tWC9Sr9wWi5ddz-x9Xi6I0pQU9VRJKlp" width=500>

In [ ]:
import tricks.md_embedding_bag as md

md.md_solver(
    torch.Tensor([100,200,300,400]),
    alpha=1.0,
    d0=128,
    round_dim=False,
)

tensor([128,  64,  43,  32])

In [ ]:
ud_emb_1 = md.PrEmbeddingBag(num_embeddings=100, embedding_dim=128, base_dim=128)
ud_emb_2 = md.PrEmbeddingBag(num_embeddings=200, embedding_dim=64, base_dim=128)
ud_emb_3 = md.PrEmbeddingBag(num_embeddings=300, embedding_dim=43, base_dim=128)
ud_emb_4 = md.PrEmbeddingBag(num_embeddings=400, embedding_dim=32, base_dim=128)

ud_emb_1, ud_emb_2, ud_emb_3, ud_emb_4

(PrEmbeddingBag(
   (embs): EmbeddingBag(100, 128, mode='sum')
   (proj): Identity()
 ),
 PrEmbeddingBag(
   (embs): EmbeddingBag(200, 64, mode='sum')
   (proj): Linear(in_features=64, out_features=128, bias=False)
 ),
 PrEmbeddingBag(
   (embs): EmbeddingBag(300, 43, mode='sum')
   (proj): Linear(in_features=43, out_features=128, bias=False)
 ),
 PrEmbeddingBag(
   (embs): EmbeddingBag(400, 32, mode='sum')
   (proj): Linear(in_features=32, out_features=128, bias=False)
 ))

\* `mode='sum'` : do element-wise addition for look-uped vectors

In [ ]:
print(sparse_indices[0])
print(sparse_offsets[0])
print(ud_emb_1(sparse_indices[0], sparse_offsets[0]).shape)
print("------------------------")
print(sparse_indices[1])
print(sparse_offsets[1])
print(ud_emb_2(sparse_indices[1], sparse_offsets[1]).shape)

tensor([ 9, 23, 29, 62, 31, 42, 43, 49])
tensor([0, 4])
torch.Size([2, 128])
------------------------
tensor([50, 62, 88, 93, 31])
tensor([0, 4])
torch.Size([2, 128])


- Run DLRM with `PrEmbeddingBag`
  - MD Trick applied to embedding table which row size is bigger than `--md-threshold`

In [ ]:
!python dlrm_s_pytorch.py --arch-sparse-feature-size=16 --arch-embedding-size="100-100" --arch-mlp-bot="13-64-16" --arch-mlp-top="16-8-4-1" \
                          --data-generation random --mini-batch-size=2 --data-size=10 \
                          --use-gpu \
                          --md-flag --md-threshold 50 --md-temperature 0.3

Unable to import mlperf_logging,  No module named 'mlperf_logging'
2023-08-07 08:34:13.470542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Unable to import onnx.  No module named 'onnx'
world size: 1, current rank: 0, local rank: 0
Using 1 GPU(s)...
time/loss/accuracy (if enabled):
Finished training it 1/5 of epoch 0, -1.00 ms/it, loss 0.207687
Finished training it 2/5 of epoch 0, -1.00 ms/it, loss 0.091614
Finished training it 3/5 of epoch 0, -1.00 ms/it, loss 0.073226
Finished training it 4/5 of epoch 0, -1.00 ms/it, loss 0.039275
Finished training it 5/5 of epoch 0, -1.00 ms/it, loss 0.009234


### **Deep Hash Embedding**
- Paper : https://arxiv.org/pdf/2010.10784.pdf

In [ ]:
!git clone https://github.com/samhsia/MP-Rec-AE.git

Cloning into 'MP-Rec-AE'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 104 (delta 29), reused 99 (delta 24), pack-reused 0
Receiving objects: 100% (104/104), 121.56 KiB | 305.00 KiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
cd MP-Rec-AE/dlrm_mprec

/content/dlrm/MP-Rec-AE/dlrm_mprec


- Before you run :
1. Open file /content/dlrm/MP-Rec-AE/dlrm_mprec/dhebag.py
2. put comment(#) on L5 : `from primesieve import nth_prime`
3. put comment(#) on L103 : `p = nth_prime(1, args_dhe["m"])`
4. remove comment(#) on L104 : `# p = self.next_prime(args_dhe['m'])`
5. Save file (ctrl+c)

- DHE archtecture :
<br>
<img src="https://drive.google.com/uc?id=1Dmm9zI62ew_b4F015mvl4aW2fAntXTH0" width=800>

In [ ]:
import dhebag as d
import numpy as np

table_sizes = [100, 200]
args_dhe = {
    "k" : 1024, # {2 ~ 2048}
    "hash_fn" : 'universal',
    "m" : 1000000,
    "seed" : 0,
    "transform" : 'uniform', # {'uniform' / 'gaussian'}
    "precision" : np.float32,
    "mlp_dims" : np.fromstring('1024-128-128', dtype=int, sep="-"), # {dhe_k - hidden_dim.- hidden_dim.- embed_vector_dim.}
    "activation" : 'mish', # {'relu' / 'sigmoid' / 'mish'}
    "batch_norm" : True,
    "num_lookups" : 2, # == len(sparse_offsets[0])
}
dhe = d.DHEBag(table_sizes, args_dhe)

dhe

<class 'numpy.float32'>


DHEBag(
  (hash_params): ParameterList(
      (0): Parameter containing: [torch.int64 of size 1024x4]
      (1): Parameter containing: [torch.int64 of size 1024x4]
  )
  (dhe_mlps): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=1024, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Mish()
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [ ]:
sparse_offsets = [torch.LongTensor([0,4]), torch.LongTensor([0,4])]
sparse_indices = [torch.LongTensor([9,23,29,62,31,42,43,49]), torch.LongTensor([50,62,88,93,31])]

result = dhe(sparse_offsets, sparse_indices)
print("result from 1st Embedding table (DHE) : ", result[0].shape)
print("result from 2nd Embedding table (DHE) : ", result[1].shape)

result from 1st Embedding table (DHE) :  torch.Size([2, 128])
result from 2nd Embedding table (DHE) :  torch.Size([2, 128])


# **Practice for DLRM TorchRec**

## **1. Installation**

In [ ]:
!pip install torch
!pip install torchrec
!pip install pyre-extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 MB 6.0 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=433668f9e92d6289efd46d4e6a49f90b2f96a68e583bd34f735465f8f037ee5b
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath


## **2. TorchRec primitive classes**

### ***KeyedJaggedTensor***
  - A KeyedJaggedTensor is a tensor with a jagged dimension which is dimension whose slices may be of different lengths. Keyed on first dimension and jagged on the last dimension.
  - `offsets=torch.tensor([0, 2, 2, 3, 4, 5, 8]) -> value[0:2]; value[2:2]; value[2:3]; value[3:4]; ...`

In [ ]:
import torch
import torchrec

#        0       1        2  <-- batch
# "f1"   [0,1] None    [2]
# "f2"   [3]    [4]    [5,6,7]
#  ^
# feature

features = torchrec.KeyedJaggedTensor(
    keys=["f1", "f2"],
    values=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7]),
    offsets=torch.tensor([0, 2, 2, 3, 4, 5, 8]),
)

print(features)

KeyedJaggedTensor({
    "f1": [[0, 1], [], [2]],
    "f2": [[3], [4], [5, 6, 7]]
})



### ***EmbeddingBagConfig & EmbeddingBagCollection***
<br>
<img src="https://drive.google.com/uc?id=1IF9Vkbp0tS1o_dg1nES0Cas5DQkjgVCV" width=600>

In [ ]:
from torchrec.modules.embedding_configs import EmbeddingBagConfig
from torchrec import EmbeddingBagCollection

table_0 = EmbeddingBagConfig(
    name="t1", embedding_dim=3, num_embeddings=10, feature_names=["f1"]
)
table_1 = EmbeddingBagConfig(
    name="t2", embedding_dim=4, num_embeddings=10, feature_names=["f2"]
)

ebc = EmbeddingBagCollection(tables=[table_0, table_1])
print(ebc)
print("input :", features)
pooled_embeddings = ebc(features)
print("output :", pooled_embeddings.values())

EmbeddingBagCollection(
  (embedding_bags): ModuleDict(
    (t1): EmbeddingBag(10, 3, mode='sum')
    (t2): EmbeddingBag(10, 4, mode='sum')
  )
)
input : KeyedJaggedTensor({
    "f1": [[0, 1], [], [2]],
    "f2": [[3], [4], [5, 6, 7]]
})

output : tensor([[ 1.4032, -1.1821, -1.8634, -1.1220, -1.0783, -0.4936, -0.4954],
        [ 0.0000,  0.0000,  0.0000, -0.3752,  1.1560,  1.6732, -1.5599],
        [-0.9413,  0.0404, -0.6278,  1.3040, -3.7977,  2.2807,  2.0177]],
       grad_fn=<CatBackward0>)


### ***DLRM & DLRMTrain***

In [ ]:
from torchrec.models.dlrm import DLRM, DLRMTrain
from torchrec.datasets.criteo import DEFAULT_INT_NAMES, DEFAULT_CAT_NAMES
from torchrec.modules.embedding_configs import EmbeddingBagConfig
from torchrec import EmbeddingBagCollection


num_embeddings_per_feature = [100*i for i in range(1, 27)] # == [100, 200, ..., 2600]
eb_configs = [
                  EmbeddingBagConfig(
                      name=f"t_{feature_name}",
                      embedding_dim=64,
                      num_embeddings=num_embeddings_per_feature[feature_idx],
                      feature_names=[feature_name],
                  )
                  for feature_idx, feature_name in enumerate(DEFAULT_CAT_NAMES)
              ]

dlrm_model = DLRM(
                    embedding_bag_collection=EmbeddingBagCollection(
                        tables=eb_configs, device=torch.device("meta")
                    ),
                    dense_in_features=len(DEFAULT_INT_NAMES),
                    dense_arch_layer_sizes=[512,256,64],
                    over_arch_layer_sizes=[512,512,256,1],
                    dense_device="cuda:0",
                  )

train_model = DLRMTrain(dlrm_model)
train_model

DLRMTrain(
  (model): DLRM(
    (sparse_arch): SparseArch(
      (embedding_bag_collection): EmbeddingBagCollection(
        (embedding_bags): ModuleDict(
          (t_cat_0): EmbeddingBag(100, 64, mode='sum')
          (t_cat_1): EmbeddingBag(200, 64, mode='sum')
          (t_cat_2): EmbeddingBag(300, 64, mode='sum')
          (t_cat_3): EmbeddingBag(400, 64, mode='sum')
          (t_cat_4): EmbeddingBag(500, 64, mode='sum')
          (t_cat_5): EmbeddingBag(600, 64, mode='sum')
          (t_cat_6): EmbeddingBag(700, 64, mode='sum')
          (t_cat_7): EmbeddingBag(800, 64, mode='sum')
          (t_cat_8): EmbeddingBag(900, 64, mode='sum')
          (t_cat_9): EmbeddingBag(1000, 64, mode='sum')
          (t_cat_10): EmbeddingBag(1100, 64, mode='sum')
          (t_cat_11): EmbeddingBag(1200, 64, mode='sum')
          (t_cat_12): EmbeddingBag(1300, 64, mode='sum')
          (t_cat_13): EmbeddingBag(1400, 64, mode='sum')
          (t_cat_14): EmbeddingBag(1500, 64, mode='sum')
         

### ***EmbeddingShardingPlanner***


In [ ]:
from torchrec.distributed.planner import *
from torchrec.distributed.sharding_plan import get_default_sharders

topology = Topology(
    local_world_size=2,
    world_size=4,
    compute_device="cuda"
)

planner = EmbeddingShardingPlanner(
        topology=topology,
        batch_size=256,
    )

plan = planner.plan(train_model, get_default_sharders())

print(plan)

module: model.sparse_arch.embedding_bag_collection

param     sharding type    compute kernel    ranks
--------  ---------------  ----------------  ------------
t_cat_0   data_parallel    dense             [0, 1, 2, 3]
t_cat_1   data_parallel    dense             [0, 1, 2, 3]
t_cat_2   data_parallel    dense             [0, 1, 2, 3]
t_cat_3   data_parallel    dense             [0, 1, 2, 3]
t_cat_4   table_wise       fused             [3]
t_cat_5   table_wise       fused             [1]
t_cat_6   table_wise       fused             [2]
t_cat_7   table_wise       fused             [0]
t_cat_8   table_wise       fused             [3]
t_cat_9   table_wise       fused             [1]
t_cat_10  table_wise       fused             [2]
t_cat_11  table_wise       fused             [0]
t_cat_12  table_wise       fused             [3]
t_cat_13  table_wise       fused             [1]
t_cat_14  table_wise       fused             [2]
t_cat_15  table_wise       fused             [0]
t_cat_16  table_wis

### ***ShardingType***

In [ ]:
from torchrec.distributed.types import ShardingType

list(ShardingType)

[<ShardingType.DATA_PARALLEL: 'data_parallel'>,
 <ShardingType.TABLE_WISE: 'table_wise'>,
 <ShardingType.COLUMN_WISE: 'column_wise'>,
 <ShardingType.ROW_WISE: 'row_wise'>,
 <ShardingType.TABLE_ROW_WISE: 'table_row_wise'>,
 <ShardingType.TABLE_COLUMN_WISE: 'table_column_wise'>]

### ***Topology***
<br>
<img src="https://drive.google.com/uc?id=1noHTFRaTgZ7_mQNIkwjkMxx1T_fdDu1x" width=500>

In [ ]:
from torchrec.distributed.planner import *

topology = Topology(
    local_world_size=2,
    world_size=4,
    compute_device="cuda"
)

topology

world_size=4 
compute_device=cuda
devices=
	device 0 DeviceHardware(rank=0, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 1 DeviceHardware(rank=1, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 2 DeviceHardware(rank=2, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 3 DeviceHardware(rank=3, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
local_world_size=2 
intra_host_bw=644245094.4 
inter_host_bw=13421772.8 

### ***HeuristicalStorageReservation***

In [ ]:
from torchrec.distributed.planner.storage_reservations import (HeuristicalStorageReservation,)
from torchrec.distributed.sharding_plan import get_default_sharders

_storage_reservation = HeuristicalStorageReservation(percentage=0.15)

storage_constraint: Topology = _storage_reservation.reserve(
                                                                topology=topology,
                                                                batch_size=256,
                                                                module=train_model,
                                                                sharders=get_default_sharders(),
                                                            )
print("<Before> : \n", topology)
print("<After>  : \n", storage_constraint)

<Before> : 
 world_size=4 
compute_device=cuda
devices=
	device 0 DeviceHardware(rank=0, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 1 DeviceHardware(rank=1, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 2 DeviceHardware(rank=2, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
	device 3 DeviceHardware(rank=3, storage=Storage(hbm=34359738368, ddr=137438953472), perf=0)
local_world_size=2 
intra_host_bw=644245094.4 
inter_host_bw=13421772.8 

<After>  : 
 world_size=4 
compute_device=cuda
devices=
	device 0 DeviceHardware(rank=0, storage=Storage(hbm=29186420404, ddr=137438953472), perf=0)
	device 1 DeviceHardware(rank=1, storage=Storage(hbm=29186420404, ddr=137438953472), perf=0)
	device 2 DeviceHardware(rank=2, storage=Storage(hbm=29186420404, ddr=137438953472), perf=0)
	device 3 DeviceHardware(rank=3, storage=Storage(hbm=29186420404, ddr=137438953472), perf=0)
local_world_size=2 
intra_host_bw=644245094.4 
inter_host_bw=13421772.

### ***EmbeddingEunmerator***

In [ ]:
from torchrec.distributed.planner.enumerators import EmbeddingEnumerator
from torchrec.distributed.planner.types import Enumerator
import pprint

_enumerator: Enumerator = EmbeddingEnumerator(
                                                topology=topology,
                                                batch_size=256,
                                              )

search_space = _enumerator.enumerate(
                                      module=train_model,
                                      sharders=get_default_sharders()
                                    )

print("length of search_space : ", len(search_space))
print("----------------------------------------------------")

print("search_space[0] : ")
pprint.pprint(search_space[0].__dict__)
print("----------------------------------------------------")

print("search_space[1] : ")
pprint.pprint(search_space[1].__dict__)

length of search_space :  416
----------------------------------------------------
search_space[0] : 
{'_module': ('model.sparse_arch.embedding_bag_collection',
             EmbeddingBagCollection(
  (embedding_bags): ModuleDict(
    (t_cat_0): EmbeddingBag(100, 64, mode='sum')
    (t_cat_1): EmbeddingBag(200, 64, mode='sum')
    (t_cat_2): EmbeddingBag(300, 64, mode='sum')
    (t_cat_3): EmbeddingBag(400, 64, mode='sum')
    (t_cat_4): EmbeddingBag(500, 64, mode='sum')
    (t_cat_5): EmbeddingBag(600, 64, mode='sum')
    (t_cat_6): EmbeddingBag(700, 64, mode='sum')
    (t_cat_7): EmbeddingBag(800, 64, mode='sum')
    (t_cat_8): EmbeddingBag(900, 64, mode='sum')
    (t_cat_9): EmbeddingBag(1000, 64, mode='sum')
    (t_cat_10): EmbeddingBag(1100, 64, mode='sum')
    (t_cat_11): EmbeddingBag(1200, 64, mode='sum')
    (t_cat_12): EmbeddingBag(1300, 64, mode='sum')
    (t_cat_13): EmbeddingBag(1400, 64, mode='sum')
    (t_cat_14): EmbeddingBag(1500, 64, mode='sum')
    (t_cat_15): Embeddin

### ***Proposers***

#### ***(1) GreedyProposer***

In [ ]:
from torchrec.distributed.planner.proposers import GreedyProposer

greedy_proposer = GreedyProposer()
greedy_proposer.load(search_space=search_space)

print("_sharding_options_by_fqn : ")
pprint.pprint(greedy_proposer._sharding_options_by_fqn)

_sharding_options_by_fqn : 
{'model.sparse_arch.embedding_bag_collection.t_cat_0': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1471090>,
 'model.sparse_arch.embedding_bag_collection.t_cat_1': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e14715d0>,
 'model.sparse_arch.embedding_bag_collection.t_cat_10': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea4a0>,
 'model.sparse_arch.embedding_bag_collection.t_cat_11': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1240f40>,
 'model.sparse_arch.embedding_bag_collection.t_cat_12': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1241630>,
 'model.sparse_arch.embedding_bag_collection.t_cat_13': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1242890>,
 'model.sparse_arch.embedding_bag_collection.t_cat_14': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13fe800>,
 'model.sparse_arch.embedding

In [ ]:
# Top 5 perf from t_cat_0
for i in range(5):
  get_perf = greedy_proposer._sharding_options_by_fqn['model.sparse_arch.embedding_bag_collection.t_cat_0'][i].__dict__['shards'][0].perf
  print(i, " : ", get_perf)

print("----------------------------------------------")

# Top 5 perf from t_cat_1
for i in range(5):
  get_perf = greedy_proposer._sharding_options_by_fqn['model.sparse_arch.embedding_bag_collection.t_cat_1'][i].__dict__['shards'][0].perf
  print(i, " : ", get_perf)

0  :  0.002743541330531024
1  :  0.02225078467940008
2  :  0.04075772254629146
3  :  0.0409698486328125
4  :  0.0409698486328125
----------------------------------------------
0  :  0.004664179739213176
1  :  0.02225078467940008
2  :  0.04075772254629146
3  :  0.0409698486328125
4  :  0.0409698486328125


In [ ]:
first_propose = greedy_proposer.propose()
print("1st propose : \n", first_propose)
greedy_proposer.feedback(partitionable=False)

second_propose = greedy_proposer.propose()
print("2nd propose : \n", second_propose)

1st propose : 
 [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1471090>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e14715d0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1473d30>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e8ee7370>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1399d20>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1399a20>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e139bc70>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13982b0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13e9360>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea7d0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea4a0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1240f40>, <torchrec.distributed.planner.types.ShardingOpt

#### ***(2) GridSearchProposer***

In [ ]:
from torchrec.distributed.planner.proposers import GridSearchProposer
from torchrec.distributed.planner.utils import prod

grid_proposer = GridSearchProposer()
grid_proposer.load(search_space=search_space)

print("_sharding_options_by_fqn : ")
pprint.pprint(grid_proposer._sharding_options_by_fqn)
print("--------------------------------------------------------------------------------------------")

print("_proposals : ")
print(grid_proposer._proposals)

_sharding_options_by_fqn : 
{'model.sparse_arch.embedding_bag_collection.t_cat_0': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1471090>,
 'model.sparse_arch.embedding_bag_collection.t_cat_1': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e14715d0>,
 'model.sparse_arch.embedding_bag_collection.t_cat_10': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea4a0>,
 'model.sparse_arch.embedding_bag_collection.t_cat_11': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1240f40>,
 'model.sparse_arch.embedding_bag_collection.t_cat_12': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1241630>,
 'model.sparse_arch.embedding_bag_collection.t_cat_13': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1242890>,
 'model.sparse_arch.embedding_bag_collection.t_cat_14': [<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13fe800>,
 'model.sparse_arch.embedding

In [ ]:
# Skipping grid search proposer as there are too many proposals
total_proposals = prod(
            [
                len(sharding_options)
                for sharding_options in grid_proposer._sharding_options_by_fqn.values()
            ]
        )


print("total_proposals : ", total_proposals) # 20282409603651670423947251286016
print("max_proposals   : ", 10000)

total_proposals :  20282409603651670423947251286016
max_proposals   :  10000


#### ***(3) UniformProposer***

In [ ]:
from torchrec.distributed.planner.proposers import UniformProposer

uni_proposer = UniformProposer()
uni_proposer.load(search_space=search_space)

print("_grouped_sharding_options :")
print(uni_proposer._grouped_sharding_options)

_grouped_sharding_options :
[[<torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1471090>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e14715d0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1473d30>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e8ee7370>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1399d20>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1399a20>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e139bc70>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13982b0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13e9360>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea7d0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e13ea4a0>, <torchrec.distributed.planner.types.ShardingOption object at 0x7c85e1242b30>, <torchrec.distributed.planner.type